In [1]:
!pip install paho-mqtt

You should consider upgrading via the 'c:\users\boomt\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install river

You should consider upgrading via the 'c:\users\boomt\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install -U numpy

Requirement already up-to-date: numpy in c:\users\boomt\appdata\local\programs\python\python38\lib\site-packages (1.21.4)


You should consider upgrading via the 'c:\users\boomt\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


# Import Library

In [1]:
import joblib
from river import tree
from river import evaluate
from river import metrics
import paho.mqtt.client as mqtt


# Activity Prediction

In [2]:
currentActivity = ''
currentActivityTimeStamp = 0
idleTime = 15

In [3]:
hoeffding_tree = joblib.load('./model.h5')
decision_tree = joblib.load('./decisionTreeModel.h5')
LABELS = {0: 'Sit', 1: 'Sleep with turning face up', 2: 'Sleep Left side'}

def predictionByDecisionTree(timeStamp, accX, accY, accZ, pitch, yaw, roll):
  global currentActivity
  global currentActivityTimeStamp
  
  X = [[accX,accY,accZ,pitch, yaw, roll]]
  y = decision_tree.predict(X)
  
  activityClass = LABELS[y[0]]

  diffTimeStamp = timeStamp - currentActivityTimeStamp
  
  if currentActivity == activityClass and diffTimeStamp > idleTime:
    alertMoveToMobile()
    currentActivity = ''
    currentActivityTimeStamp = 0
  elif currentActivity == '':
    currentActivity = activityClass
    currentActivityTimeStamp = timeStamp
  elif currentActivity != activityClass:
    currentActivity = activityClass
    currentActivityTimeStamp = timeStamp

  publishActivityToMobile(activityClass)


def predictByHoeffding(accX, accY, accZ, pitch, yaw, roll):
  X = {0:accX, 1:accY, 2:accZ, 3:pitch, 4:yaw, 5:roll}
  print(X)
  y = hoeffding_tree.predict_one(X)
  publishActivityToMobile(LABELS[y])
  

# Fall detection

In [4]:
currentSensor = {}
threshold = 3

In [5]:
def fallDetection(accX, accY, accZ, gyroX, gyroY, gyroZ):
    global currentSensor
    X = {0:accX, 1:accY, 2:accZ, 3:gyroX, 4:gyroY, 5:gyroZ}

    has_items = bool(currentSensor)
    if has_items == False:
        currentSensor = X
    else:
        
        diffAccX = abs(accX) - abs(currentSensor[0])
        diffAccY = abs(accY) - abs(currentSensor[1])
        diffAccZ = abs(accZ) - abs(currentSensor[2])
        diffGyroX = abs(gyroX) - abs(currentSensor[3])
        diffGyroY = abs(gyroY) - abs(currentSensor[4])
        diffGyroZ = abs(gyroZ) - abs(currentSensor[5])

        print(f'{diffAccX}, {diffAccY}, {diffAccZ}, {diffGyroX}, {diffGyroY}, {diffGyroZ}')
        if diffAccX > threshold or diffAccY > threshold or diffAccZ > threshold or diffGyroX > threshold or diffGyroY > threshold or diffGyroZ > threshold:
            alertFallingToMobile()

    currentSensor = X


# Handle MQTT event

In [6]:
def write_file(message):
    """
    this function write chat message to file
    """
    
    file_name = "D:\\BADS7205_RealTime\\Patient\\sleep-left-dataset.csv"
    with open(file_name, 'a') as x_file:
        x_file.write('{}\n'.format(message.encode('utf-8')))

def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result. 
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))

def on_connect(client, userdata, flags, rc):
    print("rc: " + str(rc))
 
def on_message(client, obj, msg):
    datas = msg.payload.decode("utf-8").split(',')

    if len(datas) >= 11:
        timestamp = float(datas[0])
        accX = float(datas[1])
        accY = float(datas[2])
        accZ = float(datas[3])
        gyroX = float(datas[4])
        gyroY = float(datas[5])
        gyroZ = float(datas[6])
        pitch = float(datas[7])
        yaw = float(datas[8])
        roll = float(datas[9])
        predictionByDecisionTree(timestamp, accX, accY, accZ, pitch, yaw, roll)
        fallDetection(accX, accY, accZ, gyroX, gyroY, gyroZ)
 
def on_publish(client, obj, mid):
    print("mid: " + str(mid))
 
def on_subscribe(client, obj, mid, granted_qos):
    print("Subscribed: " + str(mid) + " " + str(granted_qos))
 
def on_log(client, obj, level, string):
    print(string)

# Publish MQTT to mobile

In [7]:
import paho.mqtt.publish as publish

def alertMoveToMobile():
    mqttc.publish("alertMove", "move")

def alertFallingToMobile():
    mqttc.publish("alertFalling", "fall")

def publishActivityToMobile(activity):
    mqttc.publish("activity", activity)

# Conect MQTT

In [8]:
mqttc = mqtt.Client()
# Assign event callbacks
mqttc.on_message = on_message
mqttc.on_connect = on_connect
mqttc.on_publish = on_publish
mqttc.on_subscribe = on_subscribe


# Connect
mqttc.username_pw_set("watcharavit", "bm07739200")
mqttc.connect('qa18afea.ap-southeast-1.emqx.cloud', 15198)
 
# Start subscribe, with QoS level 0
mqttc.subscribe('sensorData', 0)

# Continue the network loop, exit when an error occurs
rc = 0
while rc == 0:
    rc = mqttc.loop()
print("rc: " + str(rc))

rc: 0
Subscribed: 1 (0,)


C:\Users\boomt\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\boomt\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


mid: 2
-0.2880401611328125, 0.1988983154296875, 0.0827789306640625, -0.21956404975547372, 0.5240874656583283, 1.9284458117031968
mid: 3
0.0379791259765625, 0.2067413330078125, -0.206695556640625, -0.7465303925521779, -0.797387714624985, -2.441334152345524
mid: 4
0.0, 0.0, 0.0, 0.21783512557579435, 0.061842062621057015, 0.20957346724215054
mid: 5


C:\Users\boomt\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\boomt\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\boomt\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\boomt\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


0.0011444091796875, -0.0033416748046875, 0.00299072265625, -0.2040841596032792, -0.06818837549871584, -0.2096123493940657
mid: 6
0.0, 0.0, 0.0, 0.055231297847153325, 0.09608019422733372, 0.17626690938518355
mid: 7


...

0.1956634521484375, 0.3993682861328125, -0.09222412109375, -0.0908939543205134, 0.11935782005128032, 1.911580012519725
mid: 757
-0.1583404541015625, -0.4542388916015625, -0.0453948974609375, 0.284302033745883, 2.6975209281473314, 5.214941326520311
mid: 758
mid: 759


C:\Users\boomt\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\boomt\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


0.1198577880859375, 0.06793212890625, 0.10382080078125, -0.01188542162891465, -2.6846847580092588, -6.919936737489281
mid: 760
0.0, 0.0, 0.0, 0.06081807718192156, 1.1193055671876921, -0.1597824748698111
mid: 761


C:\Users\boomt\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
C:\Users\boomt\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


0.00927734375, -0.0013275146484375, 0.0212554931640625, -0.16791150778116895, -0.16635622170456266, 0.1921830253239331
mid: 762
0.15435791015625, 0.050140380859375, -0.069305419921875, -0.12515818490547287, -0.9333444851185111, -0.26930231153822964
mid: 763
